In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!ls drive/MyDrive/csvData

DAI_block.csv	USDC.csv	     USDT.csv		  WBTC.csv
DAI.csv		USDC_error_info.csv  USDT_error_info.csv  WBTC_info.csv
DAI_info.csv	USDC_info.csv	     USDT_info.csv	  WETH_block.csv
USDC_block.csv	USDT_block.csv	     WBTC_block.csv	  WETH.csv


In [ ]:
import requests
import csv
from datetime import datetime

keys, info, token = None, [], "WETH"
errors = []
current_time = 1654016400
block_numbers = set()

with open("drive/MyDrive/csvData/" + token + "_block.csv", "r") as csv_file:
    reader = csv.reader(csv_file, delimiter=",")
    next(reader, None)
    for item in reader:
        num = item[1]
        if num not in block_numbers:
            block_numbers.add(num)
            query = '''
            {
                reserves(block: {number: ''' + num + '''}, where: {symbol: "''' + token + '''"}) {
                    decimals
                    totalDeposits
                    stableBorrowRate
                    variableBorrowRate
                    liquidityRate
                    utilizationRate
                    lastUpdateTimestamp
                }
            }
            '''
            response = requests.post(
                'https://api.thegraph.com/subgraphs/name/aave/protocol-v2'
                '',
                json={'query': query})
            if response.status_code != 200:
                print("Problem reading from timestamp", current_time, ":", response.status_code)
                errors.append(num)
                continue

            try:
                data = response.json()["data"]["reserves"][0]
            except Exception:
                print("Error at timestamp", current_time)
                errors.append(num)
                continue

            # if len(data) == 0:
            #     break

            print("Get block", num, "at timestamp", data["lastUpdateTimestamp"])
            
            if keys is None:
                keys = list(data.keys()) + ["blockNumber"]

            data["blockNumber"] = num
            info.append(data)

with open(token + '_info.csv', 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(info)
        

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
Get block 12450237 at timestamp 1621232045
Get block 12450246 at timestamp 1621232256
Get block 12450248 at timestamp 1621232293
Get block 12450263 at timestamp 1621232435
Get block 12450265 at timestamp 1621232458
Get block 12450283 at timestamp 1621232731
Get block 12450293 at timestamp 1621232878
Get block 12450298 at timestamp 1621232964
Get block 12450304 at timestamp 1621233004
Get block 12450308 at timestamp 1621233040
Get block 12450312 at timestamp 1621233105
Get block 12450338 at timestamp 1621233416
Get block 12450354 at timestamp 1621233709
Get block 12450382 at timestamp 1621234175
Get block 12450386 at timestamp 1621234266
Get block 12450391 at timestamp 1621234315
Get block 12450397 at timestamp 1621234432
Get block 12450403 at timestamp 1621234492
Get block 12450425 at timestamp 1621234941
Get block 12450426 at timestamp 1621234943
Get block 12450428 at timestamp 1621234956
Get block 12450430 at timestamp 16212349

In [ ]:
from google.colab import files
files.download(token + '_info.csv')

In [ ]:
errors